#RNN Fantasy Writer
José María Ibarra a01706970

In [ ]:
#librerías

import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Para crear una red recurrente capaz de escribir fantasía, es necesario alimentarla de dicho contenido. Cargamos una serie de libros para el entrenamiento.

In [ ]:
#datos

path = "/content/drive/MyDrive/got_books"
fantasy = ""
count = 0

print('Libros:')
for filename in os.listdir(path):
    if filename.endswith(".txt"):
        file_path = os.path.join(path, filename)
        count += 1
        print(count, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            fantasy += file.read()


Libros:
1 got1.txt
2 got2.txt
3 got4.txt
4 got3.txt
5 got5.txt


#Preprocesamiento

Podemos ver el vocabulario: caracteres únicos presentes en el texto y total de caracteres.

In [ ]:
#vocabulario

vocab = sorted(set(fantasy))
print(f'{len(vocab)} caracteres únicos')
print(f'{len(fantasy)} caracteres totales')

avg_letters_per_word = 4.87
print(f'{len(fantasy)/avg_letters_per_word} palabras aprox')

81 caracteres únicos
9470396 caracteres totales
1944639.8357289527 palabras aprox


El modelo deberá ser capaz de predecir el siguiente caracter según una dada secuencia de caracteres. Es necesario entonces la tokenización de cada uno de estos y la capacidad de transformarlos individualmente.

In [ ]:
#vectorización

chars = tf.strings.unicode_split(fantasy, input_encoding='UTF-8') #tokenización del texto

ids_from_chars = tf.keras.layers.StringLookup( #capa de look-up, la cual convierte cada caracter a un ID numérico
    vocabulary=list(vocab), mask_token=None)

ids = ids_from_chars(chars) #conversión a IDs

chars_from_ids = tf.keras.layers.StringLookup( #recuperamos la operación inversa, para regresar ID's a caracteres legibles
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

chars = chars_from_ids(ids)

def text_from_ids(ids): #unimos a los caracteres recuperados para generar strings
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Es necesario definir al texto como una serie de muestras de entrenamiento y categorizarlas. Las muestras son secuencias de *n* caracteres, y su categorización será la muestra recorrida un paso a la derecha, de manera que el modelo pueda identificar los caracteres siguientes para una dada secuencia.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(fantasy, 'UTF-8')) #permite obtener una lista de ID's para una el texto de entrenamiento
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) #convertimos a instancia de dataset

seq_length = 100 #definimos el tamaño de las muestras
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) #generamos batches de entrenamiento correspondientes a las secuencias

#definimos una función que nos permita obtener instancias del tipo (muestra, target), ambos siendo secuencias
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target) #generamos las instancias de entrenamiento

Antes de entrenar, es necesario generar lotes de entrenamiento y revolverlos

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

#Modelo

Generamos una subclase para un modelo

In [ ]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256 #dimensión de embedding de palabras
rnn_units = 1024 #neuronas para la capa recurrente

#subclase Model
class Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.dense1 = tf.keras.layers.Dense(vocab_size, activation='relu')
        self.dense2 = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)

        if states is None:
            states = self.gru.get_initial_state(x)

        x, states = self.gru(x, initial_state=states, training=training)
        x = self.layer_norm(x)
        x = self.dense1(x, training=training)
        x = self.dense2(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = Model( #generamos instancia de la subclasde Model
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
#observamos que el modelo se comporte como debería
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model.summary()

(64, 100, 82) # (batch_size, sequence_length, vocab_size)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  20992     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 layer_normalization (Layer  multiple                  2048      
 Normalization)                                                  
                                                                 
 dense (Dense)               multiple                  84050     
                                                                 
 dense_1 (Dense)             multiple                  6806      
                                                                 
Total params: 4052200 (15.46 MB)
Trainable params: 4052200 (15.46 MB)

#Entrenamiento

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True) #función de pérdida
model.compile(optimizer='adam', loss=loss, metrics=['accuracy']) #compilador

#callbacks para salvar iteraciones de entrenamiento
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
#entrenamiento

EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
1465/1465 [==============================] - 88s 58ms/step - loss: 1.5129 - accuracy: 0.5443
Epoch 2/30
1465/1465 [==============================] - 87s 58ms/step - loss: 1.2061 - accuracy: 0.6243
Epoch 3/30
1465/1465 [==============================] - 86s 58ms/step - loss: 1.1413 - accuracy: 0.6415
Epoch 4/30
1465/1465 [==============================] - 87s 58ms/step - loss: 1.1051 - accuracy: 0.6515
Epoch 5/30
1465/1465 [==============================] - 87s 58ms/step - loss: 1.0785 - accuracy: 0.6590
Epoch 6/30
1465/1465 [==============================] - 86s 58ms/step - loss: 1.0575 - accuracy: 0.6648
Epoch 7/30
1465/1465 [==============================] - 87s 58ms/step - loss: 1.0403 - accuracy: 0.6700
Epoch 8/30
1465/1465 [==============================] - 88s 58ms/step - loss: 1.0258 - accuracy: 0.6741
Epoch 9/30
1465/1465 [==============================] - 86s 58ms/step - loss: 1.0137 - accuracy: 0.6774
Epoch 10/30
1465/1465 [==============================] - 87s 58m

KeyboardInterrupt: ignored

In [ ]:
#guardar modelo

#model.save("fantasy_gen_v1")

#Generación de texto

In [ ]:
#generar texto

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # máscara para evitar [UNK]
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(

        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,

        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):

    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # correr modelo

    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)

    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

In [ ]:
def text_gen(prompt, length): #prompt y longitud de texto generado
  one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

  start = time.time()
  states = None
  next_char = tf.constant([prompt])
  result = [next_char]

  for n in range(length):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

  result = tf.strings.join(result)
  end = time.time()
  print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
  print('\nRun time:', end - start)


In [ ]:
#generamos con una base

text_gen('The dragon flew over the village and then', 5000)

The dragon flew over the village and then cut through the blade that shattered the bedchamber, the head blazing heavy men gasped, and joked to things the grass would need to go to the Citadel. I ought to keep him, and support Aegon’s younger mine. And fifteen years have assent that you’re not resolved to see, and Clever Lucam several thousand thousand bastard sails, and found themselves in his arms, flinging him from bone to war for them. His white rattled outer garden upon his head as hard tones of every look that made him ride face-first. Nable had settled gemstone and sword to Eriaf on the rockic trabiling shelter. When the door opened and was rested under a bedpost, painted supper and beaded verge more than a faint shit on his face. “The whinning fighters come to me and them are a glidns, I have loved you so age.” Gilly would die.

“What little of the alliances brings him a thing and bled along the grey direwolves.”

“Oh, yes.” He heard the sound of his breath and satured and packe